In [80]:
import json
import numpy as np
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, RMSprop

In [3]:
# import the pandas dataframe
data = pd.read_pickle("processed_data.pkl")

In [4]:
# change to an integer representation: 0 rather than '0'
data = data.astype("int32")

In [5]:
# separate the data into X and y data
# split using column names
X = data.iloc[:, 0:-2].to_numpy()
y = data.iloc[:, -1].to_numpy().transpose()

In [6]:
# shuffling with tensorflow
# indices = tf.range(start=0, limit=tf.shape(X)[0], dtype=tf.int32)
indices = tf.range(start=0, limit=tf.shape(X)[0])
shuffled_indices = tf.random.shuffle(indices)

2022-05-02 17:59:37.884841: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# shuffle the data for presentation to the ML models
shuffled_a = tf.gather(X, shuffled_indices)
shuffled_b = tf.gather(y, shuffled_indices)

In [14]:
# change shuffled arrays to numpy arrays
X_shuffled = np.array(shuffled_a)
y_shuffled = np.array(shuffled_b)

In [100]:
from recent_search_milo import transpose_tweeters

transposed_y_shuffled = np.empty(shape=(len(y_shuffled)))

# change the y-shuffled's to values 0-25 rather than their respective id's
for i in range(len(y_shuffled)):
    transposed_y_shuffled[i] = transpose_tweeters[y_shuffled[i]]

In [102]:
# separate training and testing data
l = int(len(X_shuffled) * 0.75)

X_train = X_shuffled[:l]
X_test = X_shuffled[l:]
y_train = transposed_y_shuffled[:l]
y_test = transposed_y_shuffled[l:]

In [230]:
# instantiate a logistic regression
log_reg = LogisticRegression(max_iter=100)

In [231]:
# fit the logistic regression, took about 6.5 minutes to train last time
log_reg.fit(X_train, y_train)

/Users/milo/opt/anaconda3/envs/data_science/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [234]:
# evaluate score
score = log_reg.score(X_test, y_test)
print(f"test accuracy of logistic regression: {score}")

test accuracy of logistic regression: 0.5716667824636976


In [214]:
model = Sequential()
model.add(Dense(60, input_shape = (14743,), activation = "relu"))
model.add(Dense(30, activation = "relu"))
model.add(layers.Dropout(0.2))
model.add(Dense(25, activation = "softmax"))
model.compile(Adam(lr = 0.01), "sparse_categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential_45"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_113 (Dense)           (None, 60)                884640    
                                                                 
 dense_114 (Dense)           (None, 30)                1830      
                                                                 
 dropout_12 (Dropout)        (None, 30)                0         
                                                                 
 dense_115 (Dense)           (None, 25)                775       
                                                                 
Total params: 887,245
Trainable params: 887,245
Non-trainable params: 0
_________________________________________________________________


In [220]:
model.fit(X_train, y_train, verbose=1, epochs=25)

Epoch 1/25
1350/1350 [==============================] - 6s 4ms/step - loss: 0.4171 - accuracy: 0.8814
Epoch 2/25
1350/1350 [==============================] - 4s 3ms/step - loss: 0.4147 - accuracy: 0.8799
Epoch 3/25
1350/1350 [==============================] - 5s 3ms/step - loss: 0.4123 - accuracy: 0.8808
Epoch 4/25
1350/1350 [==============================] - 5s 3ms/step - loss: 0.4340 - accuracy: 0.8785
Epoch 5/25
1350/1350 [==============================] - 4s 3ms/step - loss: 0.4154 - accuracy: 0.8798
Epoch 6/25
1350/1350 [==============================] - 5s 4ms/step - loss: 0.4111 - accuracy: 0.8809
Epoch 7/25
1350/1350 [==============================] - 5s 3ms/step - loss: 0.4144 - accuracy: 0.8801
Epoch 8/25
1350/1350 [==============================] - 5s 3ms/step - loss: 0.4295 - accuracy: 0.8802
Epoch 9/25
1350/1350 [==============================] - 5s 4ms/step - loss: 0.4218 - accuracy: 0.8798
Epoch 10/25
1350/1350 [==============================] - 5s 4ms/step - loss: 0.442

In [222]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy))
print('Loss: %f' % (loss))

Accuracy: 0.484541
Loss: 25.118053


In [139]:
# dictionary of most popular twitter accounts
most_popular_tweeters = {813286 : "@barackobama", 27260086 : "@justinbieber", 21447363 : "@katyperry", 
    79293791 : "@rihanna", 14230524 : "@ladygaga", 44196397 : "@elonmusk", 15846407 : "@theellenshow", 
    10228272 : "@youtube", 25365536 : "@kimkardashian", 23375688 : "@selenagomez", 428333 : "@cnnbrk", 
    783214 : "@twitter", 759251 : "@cnn", 16409683 : "@britneyspears", 11348282 : "@nasa", 21111883 : "@ddlovato", # 16
    807095 : "@nytimes", 15485441 : "@jimmyfallon", 23083404 : "@kingjames", 5402612 : "@bbcbreaking", # 20
    268414482 : "@mileycyrus", 105119490 : "@niallofficial", 96951800 : "@fcbarcelona", 627673190 : "@championsleague", 
    26257166 : "@sportscenter"}

transpose_tweeters = {813286 : 0, 27260086 : 1, 21447363 : 2, 
    79293791 : 3, 14230524 : 4, 44196397 : 5, 15846407 : 6, 
    10228272 : 7, 25365536 : 8, 23375688 : 9, 428333 : 10, 
    783214 : 11, 759251 : 12, 16409683 : 13, 11348282 : 14, 21111883 : 15, # 16
    807095 : 16, 15485441 : 17, 23083404 : 18, 5402612 : 19, # 20
    268414482 : 20, 105119490 : 21, 96951800 : 22, 627673190 : 23, 
    26257166 : 24}


reverse_transpose_tweeters = {0: 813286, 1: 27260086, 2: 21447363, 
    3:79293791, 4:14230524, 5:44196397, 6:15846407, 
    7:10228272, 8:25365536, 9:23375688, 10:428333, 
    11:783214, 12:759251, 13:16409683, 14:11348282, 15:21111883, # 16
    16:807095, 17:15485441, 18:23083404, 19:5402612, # 20
    20:268414482, 21:105119490, 22:96951800, 23:627673190, 
    24:26257166}

In [199]:
import cleantext as ct
import recent_search_milo as rsm
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import pprint

def predict_tweeter(tweet: str, model: Sequential) -> str:
    '''
    takes a tweet string from one of the twenty-five tweeters
    returns a dictionary of the tweeters' handles and their model scores
    prints the top three accounts and their scores
    '''
    
    # clean the tweet
    cleaned = ct.clean_tweet(tweet)
    # get vocabulary
    vocab = ct.get_word_set()
    # get counterizer
    cv = CountVectorizer(vocabulary=vocab)

    # get a word vector for it
    wv = ct.get_word_vector(cleaned, cv)

    # predict on the word vector
    prediction = model.predict(wv)

    # create a dictionary to hold the scores
    pred_dict = dict.fromkeys(reverse_transpose_tweeters.keys())

    # iterate through the result
    for i in range(len(prediction[0])):
        pred_dict[i] = prediction[0][i]

    # add @ keys to the dictionary
    for key in [k for k in pred_dict.keys()]:
        k = reverse_transpose_tweeters[key]
        k = most_popular_tweeters[k]
        v = pred_dict.pop(key)

        pred_dict.update({k : v})

    # create a counter object of the prediction dictionary
    cr = Counter(pred_dict)
    # get the three most likely tweeters
    highest = cr.most_common(5)

    pprint.pprint(highest)

    return pred_dict

In [244]:
# tweet from obama
pd1 = predict_tweeter("Last week, I spoke with some of the young activists who have already recognized the problem "
    "that disinformation poses to our communities and are doing their part to fix it. Take a look at our conversation:", model)

[('@barackobama', 1.0), ('@justinbieber', 0.0), ('@katyperry', 0.0)]


In [243]:
# tweet from rihanna
pd2 = predict_tweeter("Twake up and smell the billboard NYC!!!! \n\na wise woman once said," 
" “smell however the f*ck you want” 💁🏿‍♀️, \nand that wise woman is also here with great news… "
"#FENTYPARFUM is finally back baby @ http://ri-hanna.io/fentyparfum", model)

[('@selenagomez', 1.0),
 ('@rihanna', 2.9733737e-18),
 ('@jimmyfallon', 3.513508e-37)]


In [242]:
# tweet from elon musk
pd3 = predict_tweeter("Since I’ve been asked a lot:\nBuy stock in several companies that make products "
"& services that *you* believe in.\nOnly sell if you think their products & services are trending worse. "
"Don’t panic when the market does.\nThis will serve you well in the long-term.", model)

[('@nytimes', 0.99999356),
 ('@mileycyrus', 3.3356807e-06),
 ('@ddlovato', 1.6588468e-06)]


In [241]:
# tweet from cnnbrk
pd4 = predict_tweeter("A former Philadelphia police officer who fatally shot a 12-year-old boy in the back in March has been charged with first-degree murder, District Attorney Larry Krasner announced Monday.", model)

[('@cnnbrk', 1.0), ('@nytimes', 3.9457587e-10), ('@cnn', 1.3161707e-12)]


In [240]:
# tweet from rihanna
pd5 = predict_tweeter("My very first #NationalHeroesDay as a National Hero of my country Barbados! 🇧🇧🇧🇧🇧🇧 What an absolute honor to be amongst such great men and women who have come before me and held this title in commitment to our nation!", model)

[('@jimmyfallon', 0.28683314),
 ('@kingjames', 0.26641595),
 ('@theellenshow', 0.15249594)]


In [239]:
# tweet from lebron james
pd6 = predict_tweeter("Wow it's been awhile since I've seen that footage. Weight of the world on the shoulders on #ThekidfromAKRON🤴🏾", model)

[('@theellenshow', 0.7618925),
 ('@kingjames', 0.23275675),
 ('@jimmyfallon', 0.005350794)]
